In [1]:
from agents import *

ModuleNotFoundError: No module named 'ipythonblocks'

In [20]:
class ReflexPacmanAgent(Agent):
    
    def move_down(self, success=True):
        if not success:
            return
        self.location = (self.location[0] + 1, self.location[1])
    
    def move_up(self, success=True):
        if not success:
            return
        self.location = (self.location[0] - 1, self.location[1])
        
    def move_right(self, success=True):
        if not success:
            return
        self.location = (self.location[0], self.location[1] + 1)
    
    def move_left(self, success=True):
        if not success:
            return
        self.location = (self.location[0], self.location[1] - 1)
    
    def pickup(self, thing):
        if isinstance(thing, Coin):
            return True
        return False
    
    def scare(self, thing):
        if isinstance(thing, Ghost):
            return True
        return False
    
def program(percepts):
    for per in percepts: 
        if isinstance(per, Coin):
            return 'pickup'
        elif isinstance(per, Ghost):
            return 'scare'
        if isinstance(per, Bump): 
            print(" Agent bumped ")
            return random.choice(['moveright', 'moveleft', 'moveup', 'movedown'])
        else:
            return random.choice(['moveright', 'moveleft', 'moveup', 'movedown'])

In [21]:
class Coin(Thing):
    pass

class Obstacle(Thing):
    pass

class Ghost(Thing):
    pass

class Boo(Thing):
    pass

class MsPacman(Thing):
    pass

class GameMap(GraphicEnvironment):
    
    def __init__(self, agent_program, width=3, height=3, color={}):
        super().__init__(width, height, color)
        self.init_world(agent_program)

    def init_world(self, agent_program):
        ghost_prob = 0.1
        obstacle_prob = 0.2
        coin_prob = 0.7
        
        msPac = MsPacman()
        
        self.add_thing(ReflexPacmanAgent(agent_program), (self.x_start, self.y_start))
        self.add_thing(msPac, [self.x_end - 1, self.y_end - 1])

        for x in range(self.x_start, self.x_end):
            for y in range(self.y_start, self.y_end):
                if random.random() < obstacle_prob:
                    if len(self.list_things_at((x,y))) > 0 or (x == 0 and y == 0):
                        continue
                    else:
                        self.add_thing(Obstacle(), (x, y), True)
        
        for x in range(self.x_start, self.x_end):
            for y in range(self.y_start, self.y_end):
                if random.random() < ghost_prob:
                    if len(self.list_things_at((x,y))) > 0 or (x == 0 and y == 0):
                        continue
                    else:
                        self.add_thing(Ghost(), (x, y), True)
              
        for x in range(self.x_start, self.x_end):
            for y in range(self.y_start, self.y_end):
                if random.random() < coin_prob:
                    if len(self.list_things_at((x,y))) > 0 or (x == 0 and y == 0):
                        continue
                    else:
                        self.add_thing(Coin(), (x, y), True)
              
    def get_world(self):
        """Return the items in the world"""
        result = []
        x_start, y_start = (0, 0)
        x_end, y_end = self.width, self.height

        for x in range(x_start, x_end):
            row = []
            for y in range(y_start, y_end):
                row.append(self.list_things_at((x, y)))
            result.append(row)
        return result
        
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == 'moveright':
            agent.move_right()
        elif action == 'moveleft':
            agent.move_left()
        elif action == 'moveup':
            agent.move_up()
        elif action == 'movedown':
            agent.move_down()
        elif action == 'pickup':
            items = self.list_things_at(agent.location, tclass=Coin)
            if len(items) != 0:
                if agent.pickup(items[0]):
                    print('{} picked up {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
        elif action == 'scare':
            items = self.list_things_at(agent.location, tclass=Ghost)
            if len(items) != 0:
                if agent.scare(items[0]):
                    print('{} scared {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
                    
    def is_done(self):
        ''' is done when PacMan reunites with Ms. PacMan or PacMan is deas'''
        reunited = any(self.some_things_at(agent.location, tclass=MsPacman) for agent in self.agents)
        dead_pacman = not any(agent.is_alive() for agent in self.agents)
        return reunited or dead_pacman

In [23]:
from ipythonblocks import BlockGrid


gameEnv = GameMap(program,color={"ReflexPacmanAgent" : (255, 255, 0),
       "Ghost": (51, 204, 255),
       "Obstacle": (0,0,0),
       "Coin": (255, 153, 51),
       "MsPacman": (255, 153, 255)
      })

grid = BlockGrid(gameEnv.width, gameEnv.height, fill=(230, 230, 230))

def draw_grid(world):
    global grid
    grid[:] = (230, 230, 230)
    for x in range(0, len(world)):
        for y in range(0, len(world[x])):
            if len(world[x][y]):
                grid[y, x] = color[world[x][y][-1].__class__.__name__]

def step():
    global grid, gameEnv
    draw_grid(gameEnv.get_world())
    grid.show()
    gameEnv.run(10)

{}


In [24]:
step()

NameError: name 'color' is not defined

### random reflex agent :
#### P: maximize gold, minimize movement
#### E: N x N grid, obstacles,  gold and obstacle distribution unknown but actions are deterministic and env. static
#### A: move right, left, forward, backwards, climb up, climb down, collect gold, (kill ghost) 
#### S: agent cant perceive its location and if there is gold or not

In [ ]:
# TODO : implement reflex agent

### model based agent :
#### P: maximize gold, minimize movement
#### E: N x N grid, obstacles,  gold and obstacle distribution unknown but actions are deterministic and env. static
#### A: move right, left, forward, backwards, climb up, climb down, collect gold, (kill ghost) 
#### S: agent cant perceive its location and if there is gold or not

In [ ]:
# TODO : implement model agent

### utility based agent :
#### P: maximize gold, minimize movement
#### E: N x N grid, obstacles,  gold and obstacle distribution unknown but actions are deterministic and env. static
#### A: move right, left, forward, backwards, climb up, climb down, collect gold, (kill ghost) 
#### S: agent cant perceive its location and if there is gold or not

In [ ]:
# TODO : implement utility agent